In [1]:
# instalar as dependências
!chmod u=rwx,g=rx,o=rx  start.sh && ./start.sh -p "sudo_pwd" -q

./start.sh: illegal option -- q
Get:1 https://apt.releases.hashicorp.com focal InRelease [12.9 kB]             
Get:2 https://apt.releases.hashicorp.com focal/main amd64 Packages [90.4 kB]   
Get:3 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease [3611 B]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:5 https://download.docker.com/linux/ubuntu focal InRelease [57.7 kB]       
Get:6 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]          
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:8 https://download.docker.com/linux/ubuntu focal/stable amd64 Packages [31.5 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:10 https://packages.microsoft.com/ubuntu/20.04/prod focal/main armhf Packages [15.3 kB]
Get:11 https://packages.microsoft.com/ubuntu/20.04/prod focal/main amd64 Packages [215 kB]
Err:6 https://ngrok-agent.s3.amazonaws.com buster InRelease 

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "./spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [7]:
!chmod u=rwx,g=rx,o=rx  ngrok_install.sh && ./ngrok_install.sh -p "sudo_pwd" -q

./ngrok_install.sh: illegal option -- q
--2023-07-17 19:55:11--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8850130 (8.4M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.44M  3.53MB/s    in 2.4s    

2023-07-17 19:55:14 (3.53 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [8850130/8850130]

[sudo] password for thiago: ngrok
./ngrok_install.sh: line 12: ngrok: command not found
[INFO] Ngrok instalation done!


In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

#get_ipython().system_raw('./ngrok config add-authtoken YOUR_AUTHTOKEN')
get_ipython().system_raw('./ngrok http 4050 &')



Authtoken saved to configuration file: /home/thiago/.config/ngrok/ngrok.yml


In [10]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","ID":"537563abc8edbf5c3d949cf09ccde53f","uri":"/api/tunnels/command_line","public_url":"https://97cf-201-37-95-158.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":37,"gauge":0,"rate1":0.19898244031335455,"rate5":0.08494982187153469,"rate15":0.03573112767428668,"p50":160895701,"p90":30775978142.2,"p95":40831973264.1,"p99":41378740725},"http":{"count":58,"rate1":0.2594831028042488,"rate5":0.12931715538648517,"rate15":0.05548014243878217,"p50":4612200,"p90":10593410.000000017,"p95":53723004.99999992,"p99":260657800}}}],"uri":"/api/tunnels"}


## Carregamento de dados

### Dados Públicos CNPJ
#### Receita Federal

> [Empresas](https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/empresas.zip)
> 
> [Estabelecimentos](https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/estabelecimentos.zip)
> 
> [Sócios](https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/socios.zip)

[Fonte original dos dados](https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj)

---
[property SparkSession.read](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.SparkSession.read.html)

[DataFrameReader.csv(*args)](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrameReader.csv.html)



In [ ]:
import zipfile

get_ipython().system_raw('wget https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/empresas.zip -q')
zipfile.ZipFile('./empresas.zip', 'r').extractall('./data')

In [26]:
path = './data/empresas/part-0000*csv'
empresas = spark.read.csv(path, sep=';', inferSchema=True)

In [27]:
get_ipython().system_raw('pip install pandas')
import pandas
empresas.limit(5).toPandas()

/home/thiago/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2607: UserWarning: You executed the system command !pip which may not work as expected. Try the IPython magic %pip instead.
  warnings.warn(


,_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [28]:
# renomenando colunas
empresasColNames = [
    'cnpj_basico', 
    'razao_social_nome_empresarial', 
    'natureza_juridica', 
    'qualificacao_do_responsavel', 
    'capital_social_da_empresa', 
    'porte_da_empresa', 
    'ente_federativo_responsavel'
]

for index, colName in enumerate(empresasColNames):
    empresas = empresas.withColumnRenamed(f"_c{index}", colName)

empresas.columns
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [29]:
# convertendo string pra double
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f
empresas = empresas.withColumn('capital_social_da_empresa', f.regexp_replace('capital_social_da_empresa', ',', '.'))
empresas = empresas.withColumn('capital_social_da_empresa', empresas['capital_social_da_empresa'].cast(DoubleType()))
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.0,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.0,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.0,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.0,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.0,1,None


In [34]:
# convertendo string para data 
df = spark.createDataFrame([(20200924,), (20201022,), (20210215,)], ['data'])
df.toPandas()



,data
0,20200924
1,20201022
2,20210215


In [36]:
df = df.withColumn('data', f.to_date(df.data.cast(StringType()), 'yyyyMMdd'))
df.printSchema()

root
 |-- data: date (nullable = true)



In [47]:
path = './data/estabelecimentos/part-0000*'
estabelecimentos = spark.read.csv(path, sep=';', inferSchema=True)

In [48]:
estabsColNames = [
    'cnpj_basico', 
    'cnpj_ordem', 
    'cnpj_dv', 
    'identificador_matriz_filial', 
    'nome_fantasia', 
    'situacao_cadastral', 
    'data_situacao_cadastral', 
    'motivo_situacao_cadastral', 
    'nome_da_cidade_no_exterior', 
    'pais', 
    'data_de_inicio_atividade', 
    'cnae_fiscal_principal', 
    'cnae_fiscal_secundaria', 
    'tipo_de_logradouro', 
    'logradouro', 
    'numero', 
    'complemento', 
    'bairro', 
    'cep', 
    'uf', 
    'municipio', 
    'ddd_1', 
    'telefone_1', 
    'ddd_2', 
    'telefone_2', 
    'ddd_do_fax', 
    'fax', 
    'correio_eletronico', 
    'situacao_especial', 
    'data_da_situacao_especial'
]
for index, colName in enumerate(estabsColNames):
    estabelecimentos = estabelecimentos.withColumnRenamed(f"_c{index}", colName)
estabelecimentos.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,1879,1,96,1,PIRAMIDE M. C.,8,20011029,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
1,2818,1,43,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
2,3110,1,7,1,None,8,19971231,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
3,3733,1,80,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
4,4628,3,27,2,EMBROIDERY & GIFT,8,19980429,1,None,NaN,...,7075,None,None,None,None,NaN,None,None,None,NaN


In [49]:
estabelecimentos = estabelecimentos.withColumn(
            "data_situacao_cadastral",
            f.to_date(estabelecimentos.data_situacao_cadastral.cast(StringType()), 'yyyyMMdd')
            ).withColumn(
            "data_de_inicio_atividade",
            f.to_date(estabelecimentos.data_de_inicio_atividade.cast(StringType()), 'yyyyMMdd')
            ).withColumn(
            "data_da_situacao_especial",
            f.to_date(estabelecimentos.data_da_situacao_especial.cast(StringType()), 'yyyyMMdd')
        )
estabelecimentos.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,1879,1,96,1,PIRAMIDE M. C.,8,2001-10-29,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,None
1,2818,1,43,1,None,8,2008-12-31,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,None
2,3110,1,7,1,None,8,1997-12-31,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,None
3,3733,1,80,1,None,8,2008-12-31,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,None
4,4628,3,27,2,EMBROIDERY & GIFT,8,1998-04-29,1,None,NaN,...,7075,None,None,None,None,NaN,None,None,None,None


<bound method PandasConversionMixin.toPandas of DataFrame[cnpj_basico: int, cnpj_ordem: int, cnpj_dv: int, identificador_matriz_filial: int, nome_fantasia: string, situacao_cadastral: int, data_situacao_cadastral: date, motivo_situacao_cadastral: int, nome_da_cidade_no_exterior: string, pais: int, data_de_inicio_atividade: date, cnae_fiscal_principal: int, cnae_fiscal_secundaria: string, tipo_de_logradouro: string, logradouro: string, numero: string, complemento: string, bairro: string, cep: int, uf: string, municipio: int, ddd_1: string, telefone_1: string, ddd_2: string, telefone_2: string, ddd_do_fax: int, fax: string, correio_eletronico: string, situacao_especial: string, data_da_situacao_especial: date]>